In [22]:
import os, glob, shutil, json
from pathlib import Path
import SimpleITK as sitk
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import datetime
from pprint import pprint

sys.path.append("/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/src/utils")
from revert_normalisation import get_ct_normalisation_values, revert_normalisation


In [ ]:
os.environ["nnUNet_raw"] = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw"
os.environ["nnUNet_preprocessed"] = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed"
os.environ["nnUNet_results"] = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/results"

data_root = "/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1"

submission_path = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/submission"
submission_name = f'task1_{datetime.datetime.now().strftime("%Y%m%d")}_3models'


# list_data_mri = sorted(glob.glob(os.path.join(data_root, '**','mr.mha'), recursive=True))
# list_data_mask = sorted(glob.glob(os.path.join(data_root, '**','mask.mha'), recursive=True))
# print("input1 ---", len(list_data_mri), list_data_mri)
# print("input2 ---", len(list_data_mask), list_data_mask)

# Dataset AB

In [3]:
# copy files to nnUNet raw data directory
config = json.load(open("config_300.json"))
print(config)
dataset_id = config['dataset_id']
dataset_data_name = config['dataset_data_name']
dataset_shortname = config['dataset_data_name'].split('_')[-1]  # e.g., "synthrad2025_task1_MR_TH" -> "TH"
dataset_data_path = os.path.join(os.environ['nnUNet_raw'], f'Dataset{dataset_id:03d}_{dataset_data_name}')
list_data_mri = sorted(glob.glob(os.path.join(data_root, dataset_shortname, '**','mr.mha'), recursive=True))
print("input1 ---", len(list_data_mri), list_data_mri)


{'dataset_id': 300, 'dataset_data_name': 'synthrad2025_task1_MR_AB', 'dataset_target_name': 'synthrad2025_task1_CT_AB', 'data_root': '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/AB', 'preprocessing_CT': 'CT', 'preprocessing_MRI': 'MR', 'preprocessing_mask': 'no mask'}
input1 --- 27 ['/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/AB/1ABA002/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/AB/1ABA008/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/AB/1ABA010/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/AB/1ABA017/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/AB/1ABA021/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/

In [11]:


def process_file(data_path, dataset_path, modality_suffix="_0000"):
    filename = data_path.split(os.sep)[-2]
    if not filename.endswith(f'{modality_suffix}.mha'):
        filename = filename + f'{modality_suffix}.mha'
    os.makedirs(os.path.join(dataset_path, 'imagesVal'), exist_ok=True)
    shutil.copyfile(data_path, os.path.join(dataset_path, f'imagesVal/{filename}'))

with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(lambda data_path: process_file(data_path, dataset_data_path, "_0000"), list_data_mri), total=len(list_data_mri)))

# with ThreadPoolExecutor() as executor:
#     list(tqdm(executor.map(lambda data_path: process_file(data_path, dataset_data_path, "_0001"), list_data_mask), total=len(list_data_mask)))


100%|██████████| 27/27 [00:02<00:00, 12.19it/s]


In [ ]:
input_dir = os.path.join(dataset_data_path, 'imagesVal')
submission_path = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/submission"
# create submission folder with date today
submission_name = f'task1_{datetime.datetime.now().strftime("%Y%m%d")}_3models'
output_dir = os.path.join(submission_path, submission_name, f'DatasetID{dataset_id:03d}_{dataset_data_name}')
os.makedirs(output_dir, exist_ok=True)
fold = 0

In [ ]:
os.system(f"nnUNetv2_predict -d {dataset_id} -i {input_dir} -o {output_dir} -c 3d_fullres -p nnUNetPlans -tr nnUNetTrainerMRCT -f {fold}")

In [21]:

ct_plan_path = f"/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset{config['dataset_id'] + 1}_{config['dataset_target_name']}/nnUNetPlans.json"
ct_mean, ct_std = get_ct_normalisation_values(ct_plan_path)
# revert_normalisation(output_dir, ct_mean, ct_std, save_path=output_dir + "_revert_norm")
# move previous results to backup
if os.path.exists(output_dir):
    shutil.move(output_dir, output_dir + "_revert_norm/backup_normalised")

CT mean: -660.54248046875, CT std: 452.00933837890625


# Dataset HN

In [23]:
# copy files to nnUNet raw data directory
config = json.load(open("config_302.json"))
pprint(config)
dataset_id = config['dataset_id']
dataset_data_name = config['dataset_data_name']
dataset_shortname = config['dataset_data_name'].split('_')[-1]  # e.g., "synthrad2025_task1_MR_TH" -> "TH"
dataset_data_path = os.path.join(os.environ['nnUNet_raw'], f'Dataset{dataset_id:03d}_{dataset_data_name}')
list_data_mri = sorted(glob.glob(os.path.join(data_root, dataset_shortname, '**','mr.mha'), recursive=True))
print("input1 ---", len(list_data_mri), list_data_mri)


{'data_root': '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/HN',
 'dataset_data_name': 'synthrad2025_task1_MR_HN',
 'dataset_id': 302,
 'dataset_target_name': 'synthrad2025_task1_CT_HN',
 'preprocessing_CT': 'CT',
 'preprocessing_MRI': 'MR',
 'preprocessing_mask': 'no mask'}
input1 --- 34 ['/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/HN/1HNA002/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/HN/1HNA005/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/HN/1HNA017/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/HN/1HNA020/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/HN/1HNA052/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/

In [24]:
with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(lambda data_path: process_file(data_path, dataset_data_path, "_0000"), list_data_mri), total=len(list_data_mri)))

# with ThreadPoolExecutor() as executor:
#     list(tqdm(executor.map(lambda data_path: process_file(data_path, dataset_data_path, "_0001"), list_data_mask), total=len(list_data_mask)))


100%|██████████| 34/34 [00:00<00:00, 37.77it/s]


In [ ]:
input_dir = os.path.join(dataset_data_path, 'imagesVal')

output_dir = os.path.join(submission_path, submission_name, f'DatasetID{dataset_id:03d}_{dataset_data_name}')
os.makedirs(output_dir, exist_ok=True)
fold = 0
os.system(f"nnUNetv2_predict -d {dataset_id} -i {input_dir} -o {output_dir} -c 3d_fullres -p nnUNetPlans -tr nnUNetTrainerMRCT -f {fold}")


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

tile :  0.5
There are 34 cases in the source folder
[['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset302_synthrad2025_task1_MR_HN/imagesVal/1HNA002_0000.mha'], ['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset302_synthrad2025_task1_MR_HN/imagesVal/1HNA005_0000.mha'], ['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset302_synthrad2025_task1_MR_HN/imagesVal/1HNA017_0000.mha'], ['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/ra

100%|██████████████████████████████████████████| 60/60 [00:02<00:00, 21.68it/s]


sending off prediction to background worker for resampling and export
done with 1HNA002

Predicting 1HNA005:
perform_everything_on_device: True
(138, 381, 519) -0.63 4.184 float16
Reconstruction: MEAN


100%|██████████████████████████████████████████| 60/60 [00:02<00:00, 25.87it/s]


sending off prediction to background worker for resampling and export
done with 1HNA005

Predicting 1HNA017:
perform_everything_on_device: True
(115, 383, 499) -0.631 4.02 float16
Reconstruction: MEAN


...

100%|██████████████████████████████████████████| 12/12 [00:00<00:00, 26.17it/s]


sending off prediction to background worker for resampling and export
done with 1HND085

Predicting 1HND091:
perform_everything_on_device: True
(88, 271, 279) -0.637 4.11 float16
Reconstruction: MEAN


100%|██████████████████████████████████████████| 12/12 [00:00<00:00, 27.24it/s]


sending off prediction to background worker for resampling and export
done with 1HND091
(88, 268, 276) -0.6167 4.086 float16


0

In [28]:

ct_plan_path = f"/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset{config['dataset_id'] + 1}_{config['dataset_target_name']}/nnUNetPlans.json"
ct_mean, ct_std = get_ct_normalisation_values(ct_plan_path)
revert_normalisation(output_dir, ct_mean, ct_std, save_path=output_dir + "_revert_norm")
# move previous results to backup
if os.path.exists(output_dir):
    shutil.move(output_dir, output_dir + "_revert_norm/backup_normalised")

CT mean: -756.8418579101562, CT std: 460.713623046875


100%|██████████| 34/34 [00:15<00:00,  2.18it/s]


# Dataset TH

In [29]:
# copy files to nnUNet raw data directory
config = json.load(open("config_304.json"))
pprint(config)
dataset_id = config['dataset_id']
dataset_data_name = config['dataset_data_name']
dataset_shortname = config['dataset_data_name'].split('_')[-1]  # e.g., "synthrad2025_task1_MR_TH" -> "TH"
dataset_data_path = os.path.join(os.environ['nnUNet_raw'], f'Dataset{dataset_id:03d}_{dataset_data_name}')
list_data_mri = sorted(glob.glob(os.path.join(data_root, dataset_shortname, '**','mr.mha'), recursive=True))
print("input1 ---", len(list_data_mri), list_data_mri)


{'data_root': '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/TH',
 'dataset_data_name': 'synthrad2025_task1_MR_TH',
 'dataset_id': 304,
 'dataset_target_name': 'synthrad2025_task1_CT_TH',
 'preprocessing_CT': 'CT',
 'preprocessing_MRI': 'MR',
 'preprocessing_mask': 'no mask'}
input1 --- 28 ['/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/TH/1THA007/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/TH/1THA008/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/TH/1THA014/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/TH/1THA023/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/TH/1THA025/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/

In [30]:
with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(lambda data_path: process_file(data_path, dataset_data_path, "_0000"), list_data_mri), total=len(list_data_mri)))

# with ThreadPoolExecutor() as executor:
#     list(tqdm(executor.map(lambda data_path: process_file(data_path, dataset_data_path, "_0001"), list_data_mask), total=len(list_data_mask)))


100%|██████████| 28/28 [00:01<00:00, 15.02it/s]


In [31]:
input_dir = os.path.join(dataset_data_path, 'imagesVal')
submission_path = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/submission"
# create submission folder with date today
submission_name = f'task1_{datetime.datetime.now().strftime("%Y%m%d")}_3models'
output_dir = os.path.join(submission_path, submission_name, f'DatasetID{dataset_id:03d}_{dataset_data_name}')
os.makedirs(output_dir, exist_ok=True)
fold = 0
os.system(f"nnUNetv2_predict -d {dataset_id} -i {input_dir} -o {output_dir} -c 3d_fullres -p nnUNetPlans -tr nnUNetTrainerMRCT -f {fold}")


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

tile :  0.5
There are 28 cases in the source folder
[['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset304_synthrad2025_task1_MR_TH/imagesVal/1THA007_0000.mha'], ['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset304_synthrad2025_task1_MR_TH/imagesVal/1THA008_0000.mha'], ['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset304_synthrad2025_task1_MR_TH/imagesVal/1THA014_0000.mha'], ['/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/ra

100%|██████████████████████████████████████████| 48/48 [00:02<00:00, 21.44it/s]


sending off prediction to background worker for resampling and export
done with 1THA007

Predicting 1THA008:
perform_everything_on_device: True
(97, 342, 452) -0.666 2.82 float16
Reconstruction: MEAN


100%|██████████████████████████████████████████| 48/48 [00:01<00:00, 26.08it/s]


sending off prediction to background worker for resampling and export
done with 1THA008

Predicting 1THA014:
perform_everything_on_device: True
(97, 363, 468) -0.662 3.025 float16
Reconstruction: MEAN


...

100%|██████████████████████████████████████████| 80/80 [00:02<00:00, 27.54it/s]


sending off prediction to background worker for resampling and export
done with 1THB209

Predicting 1THB219:
perform_everything_on_device: True
(120, 500, 553) -0.647 2.824 float16
Reconstruction: MEAN


100%|████████████████████████████████████████| 100/100 [00:04<00:00, 24.68it/s]


sending off prediction to background worker for resampling and export
done with 1THB219
(130, 490, 548) -0.6753 2.916 float16


0

In [32]:

ct_plan_path = f"/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset{config['dataset_id'] + 1}_{config['dataset_target_name']}/nnUNetPlans.json"
ct_mean, ct_std = get_ct_normalisation_values(ct_plan_path)
revert_normalisation(output_dir, ct_mean, ct_std, save_path=output_dir + "_revert_norm")
# move previous results to backup
if os.path.exists(output_dir):
    shutil.move(output_dir, output_dir + "_revert_norm/backup_normalised")

CT mean: -753.1259765625, CT std: 409.43475341796875


  0%|          | 0/28 [00:00<?, ?it/s]

100%|██████████| 28/28 [00:28<00:00,  1.01s/it]


# final archive and submission

In [ ]:
save_path = os.path.join(submission_path, submission_name, 'Folder')
os.makedirs(save_path, exist_ok=True)

for file in glob.glob(os.path.join(submission_path, submission_name, "*_revert_norm", "*.mha")):
    new_file = os.path.join(save_path, "sct_" + os.path.basename(file))
    shutil.copyfile(file, new_file)
# zip folder
shutil.make_archive(save_path, 'zip', save_path)


'/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/submission/task1_20250708_3models/DatasetID304_synthrad2025_task1_MR_TH, Folder.zip'

In [38]:
print("Copying the line below to the local machine to download the zip file:")
print(f'rsync -avz xin015@virga.hpc.csiro.au:{save_path}.zip ./synthrad2025_submission')


Copying the line below to the local machine to download the zip file:
rsync -avz xin015@virga.hpc.csiro.au:/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/submission/task1_20250708_3models/Folder.zip ./synthrad2025_submission
